In [38]:
import pandas as pd
import numpy as np
from sklearn import datasets as skd
import os
import spacy.lang.pt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import io
import random
from nltk.stem import WordNetLemmatizer

In [39]:
nltk.download('popular', quiet=True)

True

In [40]:
info_category = os.listdir('ChatBot_data/')
info_category

['nba', 'olympics', 'sports']

In [41]:
info = skd.load_files(r"ChatBot_data", categories=info_category, shuffle= True, encoding= "ISO-8859-1")
info.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [42]:
# Selecting features 
x = info.data

# Target Value
y = info.target

In [48]:
# CountVectorize
from sklearn.feature_extraction.text import CountVectorizer
count_vector =  CountVectorizer()
count_vectorised = count_vector.fit_transform(x)

In [49]:
# TfdifTransform
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_trans = TfidfTransformer()
tfidf_transformed = tfidf_trans.fit_transform(count_vectorised)

In [50]:
# Applying an classifier
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(tfidf_transformed, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [51]:
# Prediction for new text
def prediction(text):
    new_text = [text]
    new_count_vector = count_vector.transform(new_text)
    pred = model.predict(new_count_vector)
    category = pred[0]
    return category

In [52]:
prediction("")

0

In [53]:
# def my_fun(text):
#     a = prediction(text)
#     raw = x[a].lower()
#     return raw
def sentance_tokenize(text):
    a = prediction(text)
    raw = x[a].lower()
    sent_tokens = nltk.sent_tokenize(raw)
    return sent_tokens


def word_tokenize(raw):
    a = prediction(text)
    raw = x[a].lower()
    word_tokens = nltk.word_tokenize(raw)
    return word_tokens

In [54]:
sentance_tokenize("who is amit pathania")

['national basketball association\r\nfrom wikipedia, the free encyclopedia\r\njump to navigationjump to search\r\n"nba" redirects here.',
 'for other uses, see nba (disambiguation).',
 'national basketball association (nba)\r\ncurrent season, competition or edition:\r\ncurrent sports event 2019â\x80\x9320 nba season\r\nnational basketball association logo.svg\r\nsport\tbasketball\r\nfounded\tjune 6, 1946; 73 years ago\r\n(as baa),\r\nnew york city, new york, united states[1]\r\ninaugural season\t1946â\x80\x9347\r\ncommissioner\tadam silver\r\nno.',
 "of teams\t30\r\ncountries\tunited states (29 teams)\r\ncanada (1 team)\r\nheadquarters\tnew york city\r\nmost recent\r\nchampion(s)\ttoronto raptors\r\n(1st title)\r\nmost titles\tboston celtics (17 titles)\r\ntv partner(s)\tunited states:\r\nabc/espn\r\nnba tv\r\ntnt\r\ncanada:\r\n\r\nnba tv canada\r\ntsn/tsn2\r\nsportsnet/sportsnet one\r\nofficial website\tnba.com\r\nthe national basketball association (nba) is a men's professional baske

In [55]:
# a = prediction(user_response)
# raw = data[a].lower()
# sent_tokens = nltk.sent_tokenize(raw)
# word_tokens = nltk.word_tokenize(raw)

In [56]:
# data = []
# sent_tokens = []
# word_tokens = []
# for i in range(len(info.target_names)):
#     # Convert the data into lower case
#     data.append(info.data[i].lower())
#     # Applying sentence tokenize on data
#     sent_tokens.append(nltk.sent_tokenize(data[i]))
#     # Applying word tokenize on data
#     word_tokens.append(nltk.word_tokenize(data[i]))

In [57]:
lemmer = nltk.stem.WordNetLemmatizer()
def lemTokens(token):
    return [lemmer.lemmatize(i) for i in token]
remove_punch_dict = dict((ord(punch), None) for punch in string.punctuation)
def lemm_normalize(text):
    return lemTokens(nltk.word_tokenize(text.lower().translate(remove_punch_dict)))

In [58]:
greeting_input = ("hello", "hi", "greetings", "sup", "what's up","hey",)
greeting_output = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in greeting_input:
            return random.choice(greeting_output)

In [59]:
import string
from nltk.corpus import stopwords
s_words = ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 
           'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt',
           'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve']
stop_words = stopwords.words("english")
punctuations = list(string.punctuation)
stops = stop_words + punctuations + s_words


In [60]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer = lemm_normalize, stop_words = stops)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [61]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input("User: ")
    sent_tokens = sentance_tokenize(user_response)
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
User: 
ROBO: I am sorry! I don't understand you
User: 
ROBO: I am sorry! I don't understand you
User: 
ROBO: I am sorry! I don't understand you
User: 
ROBO: I am sorry! I don't understand you
User: bye]
ROBO: I am sorry! I don't understand you
User: bye
ROBO: Bye! take care..
